In [1]:
#PRUEBA
class champ(object):
    def update_stats_base(self,stats_dict,show=False):
        for x,y in stats_dict.items():
            if x in self.stats_base.keys():
                self.stats_base[x] = y
                if show:
                    print(f"{x} = {self.stats_base[x]}")
    def update_spell_dmg(self,dmg_dict,show=False):
        for x,y in dmg_dict.items():
            if x in self.spell_dmg.keys():
                self.spell_dmg[x] = y
                if show:
                    print(f"{x} = {self.spell_dmg[x]}")
            
    def change_name(self,name):
        self.name = name
    def __init__(self):
        self.stats_base = {
            'ad': 52.0,
            'ap': 80.0,
            'mana': 100.0
        }
        self.spell_dmg = {
            'Q_dmg': 80.0 + self.stats_base['ap']*0.45 + self.stats_base['ad']*0,
            'W_dmg': 85.0 + self.stats_base['ap']*0.70 + self.stats_base['ad']*0,
            'E_dmg': 0.0 +  self.stats_base['ap']*0 +  self.stats_base['ad']*0,
            'R_dmg': 175.0 + self.stats_base['ap']*0.65 + self.stats_base['ad']*0
            #'R_dmg': 350.0 + self.stats_base['ap']*1.30 +  self.stats_base['ad']*0
        }
        # coldown and cast in seconds
        self.spell_timer = {
            'Q_cd': 6.0,
            'W_cd': 8.0,
            'E_cd': 20.0,
            'R_cd': 120.0,
            'Q_cast': 0.25,
            'W_cast': 1.46,
            'E_cast': 0.25,
            'R_cast': 0.25            
        }
        
        self.spell_cost = {
            'Q_cost': 30.0,
            'W_cost': 60.0,
            'E_cost': 70.0,
            'R_cost': 100.0
        }
        self.name = None
        
    
        
        
        

In [2]:
from scipy.optimize import linprog
import numpy as np

In [3]:
campeon = champ()
campeon.change_name('veigar')

In [4]:
print('mana',campeon.stats_base['mana'])
print('ap',campeon.stats_base['ap'])
campeon.update_stats_base({
    'mana':1000.0,
    'ap':120.0
},show=True)
print('Q_dmg',campeon.spell_dmg['R_dmg'])
print('R_dmg',campeon.spell_dmg['R_dmg'])
campeon.update_spell_dmg({
    'Q_dmg':160.0 + campeon.stats_base['ap']*0.55,
    'R_dmg':350.0 + campeon.stats_base['ap']*1.30
},show=True)

mana 100.0
ap 80.0
mana = 1000.0
ap = 120.0
Q_dmg 227.0
R_dmg 227.0
Q_dmg = 226.0
R_dmg = 506.0


In [5]:
T = 0.3
M = campeon.stats_base['mana']

costos = np.array([x for x in campeon.spell_cost.values()])
c = np.array([x for x in campeon.spell_dmg.values()])
c

array([226., 141.,   0., 506.])

In [6]:
bounds = [(0,x) for x in M/costos]
bounds

[(0, 33.333333333333336),
 (0, 16.666666666666668),
 (0, 14.285714285714286),
 (0, 10.0)]

In [7]:
A = np.diag(np.array([y for x,y in campeon.spell_timer.items() if 'cd' in x]) + 
    np.array([y for x,y in campeon.spell_timer.items() if 'cast' in x]) )
A = np.concatenate([np.vstack((costos,np.diag(A))),A])
#A = np.vstack((c,np.diag(A)))
A

array([[ 30.  ,  60.  ,  70.  , 100.  ],
       [  6.25,   9.46,  20.25, 120.25],
       [  6.25,   0.  ,   0.  ,   0.  ],
       [  0.  ,   9.46,   0.  ,   0.  ],
       [  0.  ,   0.  ,  20.25,   0.  ],
       [  0.  ,   0.  ,   0.  , 120.25]])

In [8]:
b = [T + y for x,y in campeon.spell_timer.items() if 'cd' in x]
b = np.array([M,T+sum(b)] + b)
#b = np.array([M,T+sum(b)])
b 

array([1000. ,  155.5,    6.3,    8.3,   20.3,  120.3])

In [9]:
A = A[[i for i in range(A.shape[0]) if i != 1],:]
b = b[[i for i in range(len(b)) if i != 1]]
solucion = linprog(-c, A_ub=A, b_ub=b, bounds=bounds,method = 'highs',integrality = [1,1,1,1])
#solucion = linprog(-c, A_ub=A, b_ub=b, bounds=bounds,method = 'highs')
solucion

           con: array([], dtype=float64)
 crossover_nit: -1
         eqlin:  marginals: array([], dtype=float64)
  residual: array([], dtype=float64)
           fun: -732.0
       ineqlin:  marginals: array([0., 0., 0., 0., 0.])
  residual: array([8.70e+02, 5.00e-02, 8.30e+00, 2.03e+01, 5.00e-02])
         lower:  marginals: array([0., 0., 0., 0.])
  residual: array([1., 0., 0., 1.])
       message: 'Optimization terminated successfully. (HiGHS Status 7: Optimal)'
           nit: -1
         slack: array([8.70e+02, 5.00e-02, 8.30e+00, 2.03e+01, 5.00e-02])
        status: 0
       success: True
         upper:  marginals: array([0., 0., 0., 0.])
  residual: array([32.33333333, 16.66666667, 14.28571429,  9.        ])
             x: array([1., 0., 0., 1.])

In [10]:
solucion.x

array([1., 0., 0., 1.])

In [11]:
-solucion.fun

732.0

In [12]:
campeon.spell_dmg

{'Q_dmg': 226.0, 'W_dmg': 141.0, 'E_dmg': 0.0, 'R_dmg': 506.0}

In [13]:
print(f"nombre = {campeon.name}")
print(f"stats base y daño spells")
[print(f"{x} = {y}") for x,y in campeon.stats_base.items()]
[print(f"{x} = {y}") for x,y in campeon.spell_dmg.items()]
print(f"mana gastado = {np.dot(solucion.x,costos)} de un total de {campeon.stats_base['mana']} en un tiempo de {T} segundos")
print(f"maximo daño posible {-solucion.fun} en un periodo de {T} segundos")
[print(f"Cantidad de {['Q','W','E','R'][i]} en {T} segundos: {int(solucion.x[i])}") for i in range(len(solucion.x))]

nombre = veigar
stats base y daño spells
ad = 52.0
ap = 120.0
mana = 1000.0
Q_dmg = 226.0
W_dmg = 141.0
E_dmg = 0.0
R_dmg = 506.0
mana gastado = 130.0 de un total de 1000.0 en un tiempo de 0.3 segundos
maximo daño posible 732.0 en un periodo de 0.3 segundos
Cantidad de Q en 0.3 segundos: 1
Cantidad de W en 0.3 segundos: 0
Cantidad de E en 0.3 segundos: 0
Cantidad de R en 0.3 segundos: 1


[None, None, None, None]

In [14]:
np.dot(solucion.x,costos)

130.0

In [15]:
costos

array([ 30.,  60.,  70., 100.])

In [16]:
A

array([[ 30.  ,  60.  ,  70.  , 100.  ],
       [  6.25,   0.  ,   0.  ,   0.  ],
       [  0.  ,   9.46,   0.  ,   0.  ],
       [  0.  ,   0.  ,  20.25,   0.  ],
       [  0.  ,   0.  ,   0.  , 120.25]])

In [17]:
b

array([1000. ,    6.3,    8.3,   20.3,  120.3])

In [18]:
costos

array([ 30.,  60.,  70., 100.])